In [1]:
#import tensorflowjs as tfjs
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D, Reshape, UpSampling2D, BatchNormalization, AveragePooling2D, Activation, Lambda, Conv2DTranspose, ReLU
from keras.models import Model, Sequential, load_model
from keras import backend as K
#import tensorflow as tf
from keras.regularizers import l1

in_height = 720//2
in_width =  1280//2

#out_height = 720
#out_width = 1280

out_height = in_height
out_width = in_width

#tf.compat.v1.disable_eager_execution()

Using TensorFlow backend.


In [2]:
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [10]:
encoder_model(encoder_input)

[<tf.Tensor 'encoder/z_mean/BiasAdd:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'encoder/z_log_var/BiasAdd:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'encoder/z/add:0' shape=(?, 2) dtype=float32>]

In [18]:
from keras.constraints import maxnorm
latent_dim = 200

encoder_input =  Input(shape=(in_height,in_width,3))

# Encoder Layers
encoder = Conv2D(8, (3, 3),padding='same', kernel_constraint=maxnorm(3))(encoder_input)
encoder = Activation('relu')(encoder)
encoder = AveragePooling2D((2, 2), padding='same',)(encoder)
encoder = Conv2D(16, (3, 3), padding='same', kernel_constraint=maxnorm(3))(encoder)
encoder = Activation('relu')(encoder)
encoder = AveragePooling2D((2, 2), padding='same')(encoder)

encoder = Conv2D(32, (3, 3), padding='same', kernel_constraint=maxnorm(3))(encoder)
encoder = Activation('relu')(encoder)
encoder = AveragePooling2D((2, 2), padding='same')(encoder)

shape = K.int_shape(encoder)
encoder =  Flatten(name='flatten_1')(encoder)
encoder = Dense(8, activation='relu')(encoder)

z_mean = Dense(latent_dim, name='z_mean')(encoder)
z_log_var = Dense(latent_dim, name='z_log_var')(encoder)

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

encoder_model = Model(encoder_input, [z_mean, z_log_var, z], name='encoder')
encoder_model.summary()

#encoder_model.load_weights("encoder-weights-360x640.h5")

# Decoder Layers
decoder_inputs = Input(shape=(latent_dim,), name='z_sampling')
decoder = Dense(shape[1] * shape[2] * shape[3], activation='relu')(decoder_inputs)
decoder = Reshape((shape[1], shape[2], shape[3]))(decoder)

decoder = Conv2DTranspose(32, (3, 3), padding='same',strides=2, kernel_constraint=maxnorm(3))(decoder)
decoder = Activation('relu')(decoder)

decoder = Conv2DTranspose(16, (3, 3), padding='same',strides=2, kernel_constraint=maxnorm(3))(decoder)
decoder = Activation('relu')(decoder)

decoder = Conv2DTranspose(8, (3, 3), padding='same',strides=2, kernel_constraint=maxnorm(3))(decoder)

decoder = Conv2DTranspose(3, (1, 1), padding='same')(decoder)

decoder = ReLU(max_value=255)(decoder)
decoder_model = Model(decoder_inputs,decoder)
decoder_model.summary()

outputs = decoder_model(encoder_model(encoder_input)[2])

autoencoder = Model(encoder_input, outputs, name='vae')
autoencoder.compile(optimizer='adam',  loss='mean_absolute_error',metrics=['acc'])

#autoencoder.load_weights('autoencoder-weights-model-13-360x640-360x640.h5')

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 360, 640, 3)  0                                            
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 360, 640, 8)  224         input_6[0][0]                    
__________________________________________________________________________________________________
activation_26 (Activation)      (None, 360, 640, 8)  0           conv2d_16[0][0]                  
__________________________________________________________________________________________________
average_pooling2d_16 (AveragePo (None, 180, 320, 8)  0           activation_26[0][0]              
____________________________________________________________________________________________

In [6]:
import cv2, os
import numpy as np
#import pandas as pd

train_dir = "../Datasets/DIV2K_train_HR/"
valid_dir = "../Datasets/DIV2K_valid_HR/"

X = list()
Y = list()
#files = pd.read_csv('image_data.csv').Path
train_files = os.listdir(train_dir)
valid_files = os.listdir(valid_dir)

def getImage(file):
    #‪C:\Users\mayan\Downloads\2-dog.jpg
    frame = cv2.imread(file,1)
    return [np.array([cv2.resize(frame,(in_width,in_height))]),np.array([cv2.resize(frame,(out_width,out_height))])]

def getValidation():
    x_valid,y_valid = [],[]   
    for file in valid_files:
        frame = cv2.imread(valid_dir + file,1)
        try:
            #y_valid.append(cv2.resize(frame,(out_width,out_height)))
            x_valid.append(cv2.resize(frame,(in_width,in_height)))
        except Exception as ex:
            print(ex)
    return np.array(x_valid),np.array(y_valid)


for file in train_files:
    frame = cv2.imread(train_dir + file,1)
    try:
        X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)

"""
train_files = os.listdir("C:/Users/mayan/Desktop/datasets/OutdoorSceneTrain_v2/mountain")
for file in train_files:
    frame = cv2.imread("C:/Users/mayan/Desktop/datasets/OutdoorSceneTrain_v2/mountain/" + file,1)
    try:
        height, width, channels = frame.shape
        if height >= in_height and width >= in_width:
            X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)

train_files = os.listdir("./../Datasets/flickr30k_images/")
for file in train_files:
    frame = cv2.imread( './../Datasets/flickr30k_images/'+ file,1)
    try:
        height, width, channels = frame.shape
        if height >= in_height and width >= in_width:
            X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)
"""        
x_valid,y_valid = getValidation()
X = np.array(X)

In [7]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=90,brightness_range=[0.5,1.0],zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,fill_mode="nearest")

In [24]:
from IPython.display import clear_output
from keras.optimizers import Adam, Adadelta

validation_accuracy = 0.7050989866256714
learning_rate = (1-validation_accuracy)/10000
#autoencoder.compile(optimizer=Adam(lr = learning_rate),  loss='mean_absolute_error',metrics=['acc'])

#autoencoder.compile(optimizer=Adam(lr = 2.9490101951523684e-05),  loss='mse',metrics=['acc'])
it = datagen.flow(X,batch_size=512,shuffle = True)

x_test = getImage('C:/Users/mayan/Downloads/2-dog.jpg')[0]/255;

for i in range(1000):
    X_ = next(it) 
    X_/=255
    hist = autoencoder.fit(X_, X_, 
                epochs=10,
                batch_size=32)
    clear_output() 
    
    decoded_imgs = autoencoder.predict(x_test) 
    h = autoencoder.evaluate(x_test,x_test)
        
    #print("Validation Loss",np.mean(hist.history['val_loss']))
    #print("Validation Accuracy",np.mean(hist.history['val_acc']))   
    
    print("Test Loss",h[0])
    print("Test Accuracy",h[1])
    
    cv2.imwrite("./restructured/test_image.jpg" ,decoded_imgs[0])
    cv2.imwrite("./restructured/original-test_image.jpg" ,x_test[0])
    
    #decoder_model.save("%dx%d-decoder.h5"%(out_height,out_width))
    #encoder_model.save("%dx%d-encoder.h5"%(in_height,in_width))
    autoencoder.save_weights("vae-autoencoder-weights-model-13-360x640-360x640.h5")

1/1 [==============================] - 0s 19ms/step
Test Loss 0.23255537450313568
Test Accuracy 0.25767794251441956
Epoch 1/10
376/376 [==============================] - 5s 14ms/step - loss: 0.1361 - acc: 0.4491
Epoch 2/10
 32/376 [=>............................] - ETA: 4s - loss: 0.1331 - acc: 0.4146

KeyboardInterrupt: 

In [ ]:
autoencoder.save_weights("autoencoder-weights-model-13-360x640-360x640.h5")

In [21]:
x_test = getImage('C:/Users/mayan/Desktop/tensorflow-autoencoder/static/pexels-photo-257840.jpeg')[0]/255;

decoded_imgs = autoencoder.predict(x_test)
h = autoencoder.evaluate(x_test,x_test)

print("Test Loss",h[0])
print("Test Accuracy",h[1])

cv2.imwrite("./restructured/test_image.jpg" ,decoded_imgs[0] * 255)
cv2.imwrite("./restructured/original-test_image.jpg" ,x_test[0] * 255)

1/1 [==============================] - 0s 17ms/step
Test Loss 0.2088358849287033
Test Accuracy 0.4183333218097687


True

In [ ]:
train_files = os.listdir("C:/Users/mayan/Machine Learning/Datasets/Events/")
for file in train_files:
    frame = cv2.imread("C:/Users/mayan/Machine Learning/Datasets/Events/" + file,1)
    try:
        height, width, channels = frame.shape
        if height >= in_height and width >= in_width:
            X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)

In [ ]:
h = autoencoder.evaluate(x_test,x_test)
#autoencoder.metrics_names
h

In [ ]:
"""
{'learning_rate': 2.9490101951523684e-05,
 'beta_1': 0.8999999761581421,
 'beta_2': 0.9990000128746033,
 'decay': 0.0,
 'epsilon': 1e-07,
 'amsgrad': False}
"""

autoencoder.optimizer.get_config()

In [ ]:
##Save Model
json= decoder_model.to_json()
with open("decoder.json", "w") as json_file:
    json_file.write(json)

decoder_model.save_weights("decoder.h5")

json= encoder_model.to_json()
with open("encoder.json", "w") as json_file:
    json_file.write(json)
encoder_model.save_weights("encoder.h5")

print("Saved model to disk")

In [ ]:
#x_test,y_test = next(getData(16))
x_valid,y_valid = getImage('C:/Users/mayan/Downloads/2-dog.jpg')
#x_test,y_test = getImage('dark-fantasy-wallpapers-28123-6689698.jpg')


In [ ]:
import matplotlib.pyplot as plt
import imutils, time

%matplotlib inline

num_images = 10
np.random.seed(42)

random_test_images = np.random.randint(x_valid.shape[0], size=min(num_images,x_valid.size))

encoded_imgs = encoder_model.predict(x_valid)
decoded_imgs = decoder_model.predict(encoded_imgs)

#decoded_imgs = autoencoder.predict(x_test)

plt.figure(figsize=(60, 50))

for i, image_idx in enumerate(random_test_images):
    # plot original image
    ax = plt.subplot(3, num_images, i + 1)
    plt.imshow(x_valid[image_idx])
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    cv2.imwrite("./real/%s.jpg" %time.time(),x_valid[image_idx])
    '''
    # plot encoded image
    ax = plt.subplot(3, num_images, num_images + i + 1)
    plt.imshow(encoded_imgs[image_idx].reshape(32, 16))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    '''
    # plot reconstructed image
    ax = plt.subplot(3, num_images, num_images + i + 1)
    name = time.time()
    cv2.imwrite("./restructured/%s.jpg" %name,decoded_imgs[image_idx] )
    cv2.imwrite("./restructured/original-%s.jpg" %name,x_valid[image_idx] )
    plt.imshow(decoded_imgs[image_idx]/255)
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()